# ALTEGRAD Challenge - Feature generation

*Abderrahim AIT-AZZI, Sébastien OHLEYER, Mickael SUTTON*

In [16]:
import pandas as pd
import numpy as np
import pickle
import gensim
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize
stop_words = stopwords.words('english')

## 1. WMD distance, Sent2vec, Glove

#### WMD Distance

In [17]:
def wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)

#### Sentence embedding

In [18]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

### Import embedding model

#### Glove

In [4]:
from gensim.models import KeyedVectors
# load the Stanford GloVe model
filename = './glove.42B.300d.txt.word2vec'
word_embedding_model_glove = KeyedVectors.load_word2vec_format(filename, binary=False)

FileNotFoundError: [Errno 2] No such file or directory: './glove.42B.300d.txt.word2vec'

#### W2V

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/Amine/Desktop/MVA2017/ALTEGRAD/TP3/for moodle/code/GoogleNews-vectors-negative300.bin.gz', binary=True)

### Generate features for train data

We can generate some useful features:

    -Using the fuzz package to compute some ratio of string similarity between question 1 et question 2.
    -The length of questions
    -Word embedding of the questions and compute different distances: WMD, cosine, 

In [6]:
data_train = pd.read_csv('data/train.csv', sep=',',names = ["id", "qid1", "qid2", "question1","question2","is_duplicate"])
data_train = data_train.drop(['id', 'qid1', 'qid2'], axis=1)
data_train['len_q1'] = data_train.question1.apply(lambda x: len(str(x)))
data_train['len_q2'] = data_train.question2.apply(lambda x: len(str(x)))
data_train['diff_len'] = data_train.len_q1 - data_train.len_q2
data_train['len_char_q1'] = data_train.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data_train['len_char_q2'] = data_train.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data_train['len_word_q1'] = data_train.question1.apply(lambda x: len(str(x).split()))
data_train['len_word_q2'] = data_train.question2.apply(lambda x: len(str(x).split()))
data_train['common_words'] = data_train.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
data_train['fuzz_qratio'] = data_train.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
data_train['fuzz_WRatio'] = data_train.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
data_train['fuzz_partial_ratio'] = data_train.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_train['fuzz_partial_token_set_ratio'] = data_train.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_train['fuzz_partial_token_sort_ratio'] = data_train.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_train['fuzz_token_set_ratio'] = data_train.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_train['fuzz_token_sort_ratio'] = data_train.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

#### Embedding of the questions

In [8]:
model=word_embedding_model_glove
norm_model = word_embedding_model_glove
norm_model.init_sims(replace=True)

data_train['wmd'] = data_train.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)

question1_vectors = np.zeros((data_train.shape[0], 300))
error_count = 0

for i, q in tqdm(enumerate(data_train.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

question2_vectors  = np.zeros((data_train.shape[0], 300))
for i, q in tqdm(enumerate(data_train.question2.values)):
    question2_vectors[i, :] = sent2vec(q)

#### Compute distances

In [9]:
data_train['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_train['skew_q1vec'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
data_train['skew_q2vec'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
data_train['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
data_train['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]

#### Save features for train data

In [10]:
pickle.dump(question1_vectors, open('data/q1_glove.pkl', 'wb'), -1)
pickle.dump(question2_vectors, open('data/q2_glove.pkl', 'wb'), -1)

data_train.to_csv('data/train_features_glove.csv', index=False)

### Generete features for test data

In [11]:
data_test = pd.read_csv('data/test.csv', sep=',',names = ["id", "qid1", "qid2", "question1","question2"])

In [12]:
data_test['len_q1'] = data_test.question1.apply(lambda x: len(str(x)))
data_test['len_q2'] = data_test.question2.apply(lambda x: len(str(x)))
data_test['diff_len'] = data_test.len_q1 - data_test.len_q2
data_test['len_char_q1'] = data_test.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data_test['len_char_q2'] = data_test.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data_test['len_word_q1'] = data_test.question1.apply(lambda x: len(str(x).split()))
data_test['len_word_q2'] = data_test.question2.apply(lambda x: len(str(x).split()))
data_test['common_words'] = data_test.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
data_test['fuzz_qratio'] = data_test.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
data_test['fuzz_WRatio'] = data_test.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
data_test['fuzz_partial_ratio'] = data_test.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_test['fuzz_partial_token_set_ratio'] = data_test.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_test['fuzz_partial_token_sort_ratio'] = data_test.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_test['fuzz_token_set_ratio'] = data_test.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data_test['fuzz_token_sort_ratio'] = data_test.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

In [13]:
norm_model = model
norm_model.init_sims(replace=True)
data_test['wmd'] = data_test.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)

question1_vectors = np.zeros((data_test.shape[0], 300))
error_count = 0

for i, q in tqdm(enumerate(data_test.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

question2_vectors  = np.zeros((data_test.shape[0], 300))
for i, q in tqdm(enumerate(data_test.question2.values)):
    question2_vectors[i, :] = sent2vec(q)

In [14]:
data_test['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data_test['skew_q1vec'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
data_test['skew_q2vec'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
data_test['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
data_test['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]

#### Save features for test data

In [21]:
pickle.dump(question1_vectors, open('data/q1_glove_test.pkl', 'wb'), -1)
pickle.dump(question2_vectors, open('data/q2_glove_test.pkl', 'wb'), -1)
data_test.to_csv('data/test_features_glove.csv', index=False)

### Load features

In [3]:
features_train = pd.read_csv('data/train_features_glove.csv', sep=',', encoding='latin-1')
features_test = pd.read_csv('data/test_features_glove.csv', sep=',', encoding='latin-1')
data_train = pd.read_csv('train.csv', sep=',',names = ["id", "qid1", "qid2", "question1","question2","is_duplicate"])
Y_train=data_train["is_duplicate"].values

# 2. Page Rank

In [1]:
from features_engineering.pagerank import generate_pagerank
path = "./data"

In [2]:
generate_pagerank(path)

Apply to train...
Apply to test...
Main PR generator...
Apply to train...
Writing train...
Apply to test...
Writing test...
CSV written !


# 3. Question frequency

In [3]:
from features_engineering.question_freq import generate_question_freq
path = "./data"

In [4]:
generate_question_freq(path)

Writing train features...
Writing test features...
CSV written ! see:  ./data


# 4. Intersection of questions

In [5]:
from features_engineering.question_inter import generate_question_inter
path = "./data"

In [6]:
generate_question_inter(path)

Writing train features...
Writing test features...
CSV written ! see:  ./data



# 5. K cores

In [13]:
import networkx as nx
import pandas as pd
from tqdm import tqdm

df_train = pd.read_csv('./data/train.csv', sep=',',names = ["id", "qid1", "qid2", "question1","question2","label"])
df_test = pd.read_csv('./data/test.csv', sep=',',names = ["id", "qid1", "qid2", "question1","question2"])

dfs = (df_train, df_test)

questions = []
for df in dfs:
    df['question1'] = df['question1'].str.lower()
    df['question2'] = df['question2'].str.lower()
    questions += df['question1'].tolist()
    questions += df['question2'].tolist()

graph = nx.Graph()
graph.add_nodes_from(questions)

for df in [df_train, df_test]:
    edges = list(df[['question1', 'question2']].to_records(index=False))
    graph.add_edges_from(edges)

graph.remove_edges_from(graph.selfloop_edges())

df = pd.DataFrame(data=graph.nodes(), columns=["question"])
df['kcores'] = 1

n_cores = 30
for k in tqdm(range(2, n_cores + 1)):
    ck = nx.k_core(graph, k=k).nodes()
    df['kcores'][df.question.isin(ck)] = k

print(df['kcores'].value_counts())

df.to_csv("data/question_kcores.csv", index=None)

ValueError: DataFrame constructor not properly called!

In [11]:
df_train = pd.read_csv('train.csv', sep=',',names = ["id", "qid1", "qid2", "question1","question2","label"])
df_test = pd.read_csv('test.csv', sep=',',names = ["id", "qid1", "qid2", "question1","question2"])

dfs = (df_train, df_test)

for df in dfs:
    df['question1'] = df['question1'].str.lower()
    df['question2'] = df['question2'].str.lower()
    
    q_kcores = pd.read_csv('data/question_kcores.csv', encoding="ISO-8859-1")
    
    q_kcores['question1'] = q_kcores['question']
    del q_kcores['question']
    df['q1_kcores'] = df.merge(q_kcores, 'left')['kcores']
    
    q_kcores['question2'] = q_kcores['question1']
    del q_kcores['question1']
    df['q2_kcores'] = df.merge(q_kcores, 'left')['kcores']
    
    df['q1_q2_kcores_ratio'] = (df['q1_kcores'] / df['q2_kcores']).apply(lambda x: x if x < 1. else 1./x)
    df['q1_q2_kcores_diff'] = (df['q1_kcores'] - df['q2_kcores']).apply(abs)
    df['q1_q2_kcores_diff_normed'] = (df['q1_kcores'] - df['q2_kcores']).apply(abs) / (df['q1_kcores'] + df['q2_kcores'])

df_train, df_test = dfs
df_train = df_train[['q1_kcores', 'q2_kcores', 'q1_q2_kcores_ratio', 'q1_q2_kcores_diff', 'q1_q2_kcores_diff_normed']]
df_test = df_test[['q1_kcores', 'q2_kcores', 'q1_q2_kcores_ratio', 'q1_q2_kcores_diff', 'q1_q2_kcores_diff_normed']]

FileNotFoundError: File b'data/question_kcores.csv' does not exist

#### Save features

In [7]:
df_train.to_csv('./data/df_train.csv')
df_test.to_csv('./data/df_test.csv')

NameError: name 'df_train' is not defined